In [33]:
# import os
# from PIL import Image
# from pillow_heif import register_heif_opener
# from PIL import ImageFile

# # Cho phép xử lý tệp bị lỗi hoặc không đầy đủ
# ImageFile.LOAD_TRUNCATED_IMAGES = True

# # Đăng ký HEIF/HEIC opener với Pillow
# register_heif_opener()

# def convert_heic_to_png(heic_file, png_file):
#     """Chuyển đổi tệp HEIC sang PNG"""
#     try:
#         image = Image.open(heic_file)  # Mở tệp HEIC
#         image.save(png_file, "PNG")   # Lưu dưới dạng PNG
#         print(f"Converted {heic_file} to {png_file}")
#     except Exception as e:
#         print(f"Failed to convert {heic_file}: {e}")

# # Thư mục chứa tệp HEIC
# file_path = "./DATA"

# # Duyệt qua tất cả các tệp trong thư mục
# for file in os.listdir(file_path):
#     if file.lower().endswith(".heic"):  # Kiểm tra đuôi tệp
#         heic_file = os.path.join(file_path, file)
#         png_file = os.path.splitext(heic_file)[0] + ".png"

#         # Kiểm tra nếu file PNG đã tồn tại
#         if os.path.exists(png_file):
#             print(f"Skipping {file}, {png_file} already exists.")
#             continue

#         # Chuyển đổi nếu file PNG chưa tồn tại
#         convert_heic_to_png(heic_file, png_file)
#     else:
#         print(f"Skipping {file}, not a HEIC file.")

# print("Done")


In [34]:
# !jupyter nbconvert --to script run.ipynb

In [35]:
# !python mmocr_sam.py --inputs ./data/DataFood/IMG_3455.png --outdir ./OUTPUT --device cpu

In [ ]:
import os
import shutil
import cv2
import torch
import numpy as np
from underthesea import word_tokenize
from mmocr.apis.inferencers import MMOCRInferencer
import warnings

warnings.filterwarnings("ignore")


In [ ]:
import numpy as np
from underthesea import word_tokenize
from mmocr.apis.inferencers import MMOCRInferencer
import warnings

warnings.filterwarnings("ignore")


In [ ]:
import numpy as np
from underthesea import word_tokenize
from mmocr.apis.inferencers import MMOCRInferencer
import warnings

warnings.filterwarnings("ignore")


In [37]:
# Hàm scale ảnh
def scale_image(image, scale_factor):
    """Scale image by a given factor."""
    height, width = image.shape[:2]
    new_width = int(width * scale_factor)
    new_height = int(height * scale_factor)
    scaled_image = cv2.resize(
        image, (new_width, new_height), interpolation=cv2.INTER_LINEAR
    )
    return scaled_image

In [38]:
# Hàm tính khoảng cách giữa hai bounding boxes
def calculate_distance(box1, box2):
    """Calculate the Euclidean distance between the centers of two bounding boxes."""
    center1 = ((box1[0] + box1[2]) / 2, (box1[1] + box1[3]) / 2)
    center2 = ((box2[0] + box2[2]) / 2, (box2[1] + box2[3]) / 2)
    return np.linalg.norm(np.array(center1) - np.array(center2))

In [39]:
# Hàm gom cụm văn bản theo dòng và khoảng cách
def group_texts_by_proximity(detected_texts, line_threshold=20, distance_threshold=50):
    """
    Group texts by proximity based on the vertical (y) alignment and horizontal distance.
    
    Args:
        detected_texts: List of tuples [(text, (x_min, y_min, x_max, y_max)), ...].
        line_threshold: Maximum y-difference to group texts into the same line.
        distance_threshold: Maximum horizontal distance to group texts within the same line.
        
    Returns:
        List of grouped texts as sentences.
    """
    # Sắp xếp bounding boxes theo tọa độ y_min (dòng trên cùng trước)
    detected_texts = sorted(detected_texts, key=lambda x: x[1][1])  # x[1][1] là y_min

    lines = []
    current_line = []

    # Gom nhóm bounding boxes theo dòng
    for text, box in detected_texts:
        if not current_line:
            current_line.append((text, box))
            continue

        _, prev_box = current_line[-1]
        if abs(box[1] - prev_box[1]) < line_threshold:  # So sánh y_min của box
            current_line.append((text, box))
        else:
            lines.append(current_line)
            current_line = [(text, box)]

    if current_line:
        lines.append(current_line)

    # Gom cụm văn bản trong mỗi dòng
    grouped_texts = []
    for line in lines:
        line = sorted(line, key=lambda x: x[1][0])  # Sắp xếp theo x_min (trái sang phải)
        current_group = []
        sentence = ""

        for text, box in line:
            if not current_group:
                current_group.append((text, box))
                sentence = text
                continue

            _, prev_box = current_group[-1]
            if box[0] - prev_box[2] < distance_threshold:  # So sánh khoảng cách x_min với x_max trước đó
                current_group.append((text, box))
                sentence += " " + text
            else:
                grouped_texts.append(sentence)
                current_group = [(text, box)]
                sentence = text

        if sentence:
            grouped_texts.append(sentence)

    return grouped_texts

In [40]:
# Hàm xử lý chính tả
def correct_vietnamese(text):
    """Correct Vietnamese text using underthesea."""
    tokens = word_tokenize(text)
    return " ".join(tokens)


In [41]:
# Hàm xử lý ảnh OCR
def process_image(image_path, output_dir, mmocr_inferencer, scale_factor):
    """Process a single image using MMOCR, group texts, and correct them."""
    # Đọc ảnh
    image = cv2.imread(image_path)
    if image is None:
        print(f"Cannot read image: {image_path}")
        return

    # Scale ảnh
    scaled_image = scale_image(image, scale_factor)

    # Thực hiện OCR
    results = mmocr_inferencer(scaled_image, save_vis=True, out_dir=output_dir)

    # Kiểm tra kết quả
    if "predictions" not in results:
        print(f"No predictions found for {image_path}")
        return

    # Lấy kết quả văn bản và bounding boxes
    detected_texts = []
    for prediction in results["predictions"]:
        if "rec_texts" in prediction and "det_polygons" in prediction:
            for text, polygon in zip(prediction["rec_texts"], prediction["det_polygons"]):
                try:
                    # Chuyển polygon thành bounding box
                    points = [(polygon[i], polygon[i + 1]) for i in range(0, len(polygon), 2)]
                    x_min = min(point[0] for point in points)
                    y_min = min(point[1] for point in points)
                    x_max = max(point[0] for point in points)
                    y_max = max(point[1] for point in points)
                    box = (x_min, y_min, x_max, y_max)
                    detected_texts.append((text, box))
                except Exception as e:
                    print(f"Error processing polygon: {polygon}, Error: {e}")

    # Bỏ qua ảnh không có văn bản
    if not detected_texts:
        print(f"No text detected in {image_path}")
        return

    # Gom cụm văn bản
    grouped_texts = group_texts_by_proximity(detected_texts, line_threshold=20, distance_threshold=50)

    # Sửa lỗi chính tả
    corrected_texts = [correct_vietnamese(sentence) for sentence in grouped_texts]

    # Ghi kết quả văn bản vào file txt
    if corrected_texts:
        output_text_file = os.path.join(
            output_dir,
            os.path.basename(image_path)
            .replace(".jpg", ".txt")
            .replace(".png", ".txt"),
        )
        with open(output_text_file, "w", encoding="utf-8") as f:
            f.write("\n".join(corrected_texts))
        print(f"Text saved to: {output_text_file}")

In [42]:
# Xóa dữ liệu cũ trong thư mục OUTPUT
def prepare_output_dir(output_dir):
    """Clear old output directory and create a new one."""
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

In [43]:
# Hàm chính để xử lý tất cả ảnh trong thư mục
def process_all_images(input_dir, output_dir, mmocr_inferencer, scale_factor):
    """Process all images in a directory."""
    prepare_output_dir(output_dir)
    image_files = [
        os.path.join(input_dir, f)
        for f in os.listdir(input_dir)
        if f.endswith((".jpg", ".png"))
    ]
    for image_file in image_files:
        process_image(image_file, output_dir, mmocr_inferencer, scale_factor)
    print("Processing complete. Results saved in:", output_dir)

In [44]:
# Cấu hình MMOCR Inferencer
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
mmocr_inferencer = MMOCRInferencer(
    det="mmocr_dev/configs/textdet/dbnetpp/dbnetpp_swinv2_base_w16_in21k.py",
    det_weights="checkpoints/mmocr/db_swin_mix_pretrain.pth",
    rec="mmocr_dev/configs/textrecog/abinet/abinet_20e_st-an_mj.py",
    rec_weights="checkpoints/mmocr/abinet_20e_st-an_mj_20221005_012617-ead8c139.pth",
    device=DEVICE,
)

Loads checkpoint by local backend from path: checkpoints/mmocr/db_swin_mix_pretrain.pth
11/28 23:47:16 - mmengine - INFO - Delete `relative_position_index` and `relative_coords_table` since we always re-init these params according to the `window_size`, which might cause unwanted but unworried warnings when loading checkpoint.
The model and loaded state dict do not match exactly

missing keys in source state_dict: backbone.stages.0.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.0.attn.w_msa.relative_position_index, backbone.stages.0.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.0.blocks.1.attn.w_msa.relative_position_index, backbone.stages.1.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.0.attn.w_msa.relative_position_index, backbone.stages.1.blocks.1.attn.w_msa.relative_coords_table, backbone.stages.1.blocks.1.attn.w_msa.relative_position_index, backbone.stages.2.blocks.0.attn.w_msa.relative_coords_table, backbone.stages.2.blocks.0

In [45]:
# Cấu hình thư mục và mô hình
INPUT_DIR = "./data/Test"  # Thư mục chứa ảnh đầu vào
OUTPUT_DIR = "./OUTPUT"  # Thư mục lưu kết quả
SCALE_FACTOR = 1.5  # Hệ số scale ảnh


In [46]:
process_all_images(INPUT_DIR, OUTPUT_DIR, mmocr_inferencer, SCALE_FACTOR)

Text saved to: ./OUTPUT\IMG_3464.txt


Text saved to: ./OUTPUT\IMG_3529.txt
Processing complete. Results saved in: ./OUTPUT
